In [1]:
import pandas as pd
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
urlav = "https://raw.githubusercontent.com/olagit/Structure-vs-properties-of-compounds/main/data/Avalon.csv"
avalon = pd.read_csv(urlav)

In [3]:
avalon = avalon.drop(["index", "smiles"], axis=1)
avalon

,logP,fp0,fp1,fp2,fp3,fp4,fp5,fp6,fp7,fp8,...,fp502,fp503,fp504,fp505,fp506,fp507,fp508,fp509,fp510,fp511
0,-0.770,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,-3.300,0,0,0,0,0,1,0,0,1,...,0,0,1,1,0,1,0,0,0,0
2,-2.060,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-7.870,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,-1.330,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,-1.710,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
1124,0.106,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
1125,-3.091,1,1,1,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1126,-3.180,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X = avalon.iloc[:,1:]
y = avalon.iloc[:,0]
X = torch.tensor(X.values)
y = torch.tensor(y.values)

In [29]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

from typing import List, Tuple

from mldd.metrics import mae, rmse, rocauc, r_squared
from mldd.data import *


def train(X_train, y_train, X_valid, y_valid):
    # hyperparameters definition
    hidden_size = 512
    epochs = 50
    batch_size = 64
    learning_rate = 0.0001
    
    # model preparation
    model = nn.Sequential(nn.ReLU(),
                          nn.Softmax(dim=1),
                          nn.Linear(1024, 1))
                          
    model.train()
    
    # data preparation
    dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train.reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # training loop
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    loss_fn = nn.MSELoss()
    for epoch in trange(1, epochs + 1, leave=False):
        for X, y in tqdm(loader, leave=False):
            model.zero_grad()
            preds = model(X)
            loss = loss_fn(preds, y)
            loss.backward()
            optimizer.step()
    return model


def predict(model, X_test, y_test):
    # hyperparameters definition
    # (but this doesn't change the training results, it's only to optimize the eval speed)
    batch_size = 64

    # data preparation
    dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test.reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    # evaluation loop
    preds_batches = []
    with torch.no_grad():
        for X, y in tqdm(loader):
            preds = model(X)
            preds_batches.append(preds.cpu().detach().numpy())
    preds = np.concatenate(preds_batches)
    return preds

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.5, random_state=123)

for _ in range(X_train.shape[0]):
        
    # training
    model = train(X_train, y_train, X_valid, y_valid)
    
    # evaluation
    predictions = predict(model, X_test, y_test)
    
    rmse_score = rmse(y_test, predictions)
    mae_score = mae(y_test, predictions)
#     r2_score = r_squared(y_test, predictions)
    scores.append([rmse_score, mae_score])
    
    break  # can be removed to get results on all folds
scores = np.array(scores)
print('RMSE, MAE = ' + \
      ', '.join(f'{mean:.2f}±{std:.3f}' for mean, std in zip(scores.mean(axis=0), scores.std(axis=0))))

print(predictions)

TypeError: expected TensorOptions(dtype=float, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)) (got TensorOptions(dtype=long long, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)))